# Conditionnement avec SageMath

In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 4 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Evaluation du conditionnement de matrices à valeurs rationnelles avec SageMath 
#    et calcul d'inverses - attention à l'évaluation du conditionnement avec SciPy ! 
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import warnings

## Matrice de Hilbert

### Conditionnement de la matrice de Hilbert

Une matrice de Hilbert est une matrice carrée de terme général :

$$ H_{ij} = \frac{1}{i+j-1} $$

La matrice de Hilbert de taille 6 s'écrit :


$$\begin{pmatrix}
 1 & \displaystyle \frac{1}{2} & \displaystyle \displaystyle \frac{1}{3} &\displaystyle  \frac{1}{4} & \displaystyle \frac{1}{5} & \displaystyle  \frac{1}{6} \\
\displaystyle \frac{1}{2} & \displaystyle \frac{1}{3} & \displaystyle \frac{1}{4} &\displaystyle  \frac{1}{5} & \displaystyle \frac{1}{6} & \displaystyle  \frac{1}{7} \\
\displaystyle \frac{1}{3} & \displaystyle \frac{1}{4} & \displaystyle \frac{1}{5} &\displaystyle  \frac{1}{6} & \displaystyle \frac{1}{7} & \displaystyle  \frac{1}{8} \\
\displaystyle \frac{1}{4} & \displaystyle \frac{1}{5} &\displaystyle \frac{1}{6} &\displaystyle  \frac{1}{7} & \displaystyle \frac{1}{8} & \displaystyle  \frac{1}{9} \\
\displaystyle \frac{1}{5} & \displaystyle \frac{1}{6} & \displaystyle \frac{1}{7} &\displaystyle  \frac{1}{8} & \displaystyle \frac{1}{9} & \displaystyle  \frac{1}{10} \\
\displaystyle \frac{1}{6} & \displaystyle \frac{1}{7} & \displaystyle \frac{1}{8} &\displaystyle  \frac{1}{9} & \displaystyle \frac{1}{10} & \displaystyle  \frac{1}{11} \\
\end{pmatrix}$$

In [ ]:
def cond_hilbert(n):
    A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
    print("-----------------------------------------------------------------------")
    print(f"Matrice de Hilbert de taille {n} :")
    print(A.str(unicode=True, character_art=True))
    print(f"Inverse de la matrice de Hilbert taille {n} :")
    if (n<8):
        print((A^-1).str(unicode=True, character_art=True))
    c_inf = A.norm(Infinity) * (A^-1).norm(Infinity)
    print(f"Conditionnement associé à la norme infinie    : {c_inf}")
    c_2 = A.norm(2) * (A^-1).norm(2)
    print(f"Conditionnement associé à la norme 2          : {c_2}")
    print(f"Conditionnement associé à la norme 2 (linalg) : {np.linalg.cond(scipy.linalg.hilbert(n))}")

    #return A.norm(Infinity) * (A^-1).norm(Infinity), A.norm(2) * (A^-1).norm(2)

for i in range(2,15): 
    #print(cond_hilbert(i))    
    cond_hilbert(i)
    print()

Le lecteur pourra constater que : 
- le conditionnement augmente démesurément vite avec la taille $n$
- le calcul du conditionnement basé sur la **linalg** est correct jusqu'à $n=12$ et faux ensuite (basé sur une évaluation des valeurs propres qui devient grossièrement fausse du fait du conditionnement de la matrice - voir ci-dessous)
- évidemment, le conditionnement dépend de la norme utilisée

### Conditionnement et valeurs propres 

Les outils standard de calcul numérique en double précision ne peuvent pas évaluer correctement les valeurs propres de la matrice et par conséquent le conditionnement au sens de la norme 2, au delà de n=13 ! 

In [ ]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt

def cond_hilbert(n):
    A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
    print("-----------------------------------------------------------------------")
    c_inf = A.norm(Infinity) * (A^-1).norm(Infinity)
    print(f"Conditionnement associé à la norme infinie    : {c_inf}")
    c_2 = A.norm(2) * (A^-1).norm(2)
    print(f"Conditionnement associé à la norme 2          : {c_2}")
    print(f"Conditionnement associé à la norme 2 (linalg) : {np.linalg.cond(scipy.linalg.hilbert(n))}")
    
%matplotlib inline

n = 20

b=scipy.linalg.hilbert(n)
BB = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])

#print(f"Conditionnement de la matrice de Hilbert de taille {n} : {np.linalg.cond(b)}")
#print("calculé avec linalg de numpy")

cond_hilbert(n)
print()
print()
eig_val = np.linalg.eigvals(b)
#eig_val, inverse = np.linalg.eigh(b)
print("Valeurs propres evaluées avec NumPy Linalg: ")
print(eig_val)
print()
ex_eig_val = BB.eigenvalues()
new_ex_eig_val = ex_eig_val[::-1]
print("Valeurs propres evaluées avec SageMath: ")
print(new_ex_eig_val)
#print(ex_eig_val)

#print(A^-1)
#print(inverse)

# Taille de la figure
plt.figure(figsize=[16, 8])
# titre de la figure
plt.title("Représentation des valeurs propres")
# titre des axes
plt.xlabel('numéro de la valeur propre')
plt.ylabel('Log des valeurs propres')
# limites des axes
#plt.axis([0, n, np.min(np.abs(eig_val.real)), 10.])
plt.axis([0, n, 1.e-29, 10.])
plt.bar(1.+np.arange(eig_val.size), np.abs(eig_val.real), width = 0.1, log = True, color = 'blue')
#plt.bar(1.+np.arange(eig_val.size), np.abs(ex_eig_val), width = 0.1, log = True, color = 'red')
plt.bar(1.+np.arange(eig_val.size), np.abs(new_ex_eig_val), width = 0.1, log = True, color = 'red')
couleur = ['NumPy','SageMath']
plt.legend(couleur,loc=1)
plt.show()

Au delà de la quinzième valeur propre, l'évaluation par NumPy est fausse et impacte le conditionnement comme on le voit bien en comparant à l'évaluation de SageMath qui capture le spectre de manière correcte.

### Perturbation de la matrice et résolution (SageMath!)

L'idée est ici d'illustrer le mauvais conditionnement de la matrice de Hibert lors de la résolutin du système linéaire correspondant en évaluant l'impact d'une perturbation de petite taille sur les données du problème.  Le second membre est fixé de telle manière à ce que la solution exacte soit constituée d'un vecteur de $1$. Le second membre et la solution sont donc bien des vecteurs de nombres rationels.

Nous  commençons par une perturbation rationnelle de petite taille (1/1000000) sur la matrice de Hilbert et effectuons une résolution exacte dans le corps des rationnel pour le problème perturbé. L'impact du mauvais conditionnement sur la solution apparaît alors clairement dès que la taille de la matrice de Hilbert atteint une dizaine. Nous ne présentons les solutions exactes du problème perturbé que dans les cas où $n$ reste de petite taille pour des questions de lisibilité.

In [ ]:
def diff_hilbert_mat(n):
    print("-----------------------------------------------------------------------")
    print(f"Matrice de Hilbert de taille {n} :")
    x = vector(QQ,[1 for i in range(0,n)])
    A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
    #print(i,"conditionnement",A.norm(Infinity) * (A^-1).norm(Infinity), A.norm(2) * (A^-1).norm(2))
    print("Conditionnement associé à la norme infinie :",A.norm(Infinity) * (A^-1).norm(Infinity))

    
    y = A*x
    print("On perturbe le dernier élément diagonal de : ", 1/(10^6)) 
    A[n-1,n-1] = (1/(2*n-1))*(1+1/(10^6))   # perturbe la matrice
    #print("Conditionnement associé à la norme infinie :",A.norm(Infinity) * (A^-1).norm(Infinity))

    s = A\y
    if (n<=5):
        print("Solution exacte du système initiale  :", x)
        print("Solution exacte du système perturbée :", s)
    err = max(abs(float(s[i]-x[i])) for i in range(0,n))
    print("Norme infinie de l'ecart entre les deux solutions : ", err)

for i in range(1,20):
    #print(diff_hilbert(i))
    diff_hilbert_mat(i)
    print()

### Perturbation du second membre et résolution (SageMath!) 

Dans un second temps, on garde le même problème de départ mais on perturbe le second membre uniquement, dans le même esprit, c'est-à-dire avec une perturbation rationnelle d'amplitude $1/1000000$.

In [ ]:
def diff_hilbert_b(n):
    print("-----------------------------------------------------------------------")
    print(f"Matrice de Hilbert de taille {n} :")
    x = vector(QQ,[1 for i in range(0,n)])
    A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
    #print(n,"conditionnement",A.norm(Infinity) * (A^-1).norm(Infinity), A.norm(2) * (A^-1).norm(2))
    print("Conditionnement associé à la norme infinie :",A.norm(Infinity) * (A^-1).norm(Infinity))

    y = A*x
    print("On perturbe le dernier élément du second membre de : ", 1/(10^6)) 

    y[n-1] *= (1+1/(1000000)) #  perturbation du second membre
    s = A\y
    if (n<=6):
        print("solution exacte du système initial  :", x)
        print("solution exacte du système perturbé :", s)
    err = max(abs(float(s[i]-x[i])) for i in range(0,n))
    print("Norme infinie de l'ecart entre les deux solutions : ", err)
    return max(abs(float(s[i]-x[i])) for i in range(0,n))

for i in range(1,15):
    diff_hilbert_b(i)
    print()

Dans ce cadre, on observe que dès $n=6$, la solution perturbée est très différente de la solution initiale. Rappelons que l'on effectue ici un calcul exact, ce qui permet d'évaluer le conditionnement mathématique du problème de résolution d'un système linéaire, où la matrice est une matrice de Hilbert.

### Un cas 4x4 mal conditionné - G. Wanner.

Nous revenons ici sur l'exemple de G. Wanner, où on voit la précision se dégrader sur la résolution d'un système linéaire 4x4 ! Dans le même esprit que précédemment mais en observant l'impact de la réprésentation en machine avec des flottants simple et double précision.

In [ ]:
import numpy as np
import scipy.linalg as LA

n=4
A = matrix(QQ, [[1/(i+j) for j in [1..n]] for i in [1..n]])

#print(A,"conditionnement",A.norm(Infinity) * (A^-1).norm(Infinity), A.norm(2) * (A^-1).norm(2))
b = vector(QQ, [3511/13860,277/1540,40877/291060,3203/27720])

print("Résolution du système Ax = b")
print("avec A =")
print(A)
print("et b = ", b)

b64 = np.array(b)
b32 = b64.astype(np.float32)
A64 = np.array(A)
A32 = A64.astype(np.float32)

x = A.solve_right(b)
print("\nSolution exacte :", x)
print("\nConditionnement en norme 2",A.norm(2) * (A^-1).norm(2))

x64 = LA.solve(A,b)
np.set_printoptions(precision=15)
print("\nSolution numerique (précision de 64 bits):", x64)
print("Résidu |Ax-b| = ", np.dot(A64,x64)-b64)

x32 = LA.solve(A32,b32)
np.set_printoptions(precision=7)
print("\nSolution numerique (précision de 32 bits):", x32)
print("Résidu |Ax-b| = ", np.dot(A32,x32)-b32)

## Matrice de Vandermonde

le but ici est d'observer le conditionnement de la matrice de Vandermonde et son impact sur la résolution du système linéaire correspondant pour un choix particulier de second membre.

Les erreurs commises du fait de la réprésentation des nombres réels en machine en simple et double précision sont évaluées en utilisant une résolution exacte avec SageMath. On montre effectivement que les erreurs sont directement liées au conditionnement de la matrice si l'on utilise une résolution avec SciPy.


In [ ]:
import numpy as np

def vandermonde(n):
#    X = np.array([i for i in range(1,n+1)])
    X = np.array([i for i in range(1,n+1)])
    A = np.array([X**i for i in range(n)]).T
    return A

np.set_printoptions(linewidth=120)
print(vandermonde(10))

### Conditionnement et impact sur la résolution

In [ ]:
warnings.filterwarnings('ignore')

def vandermonde2(x): # ici on attend une liste ou un intervalle
    n = len(x)
    x = np.array(x)
    return np.vstack([x**i for i in range(n)]).T

liste = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
#liste = [1,1/2,1/3,1/4,1/5,1/6,1/7,1/8,1/9,1/10,1/11,1/12,1/13,1/14]

for i in range(4,13):
    print("-------------------------------------------------")
    print("Matrice de vandermonde de taille : ", i)
    A = matrix(QQ,vandermonde2(liste[1:i+1]))
    bn = np.ones(i)/2
    #cond = A.norm(Infinity) * (A^-1).norm(Infinity)
    cond = A.norm(2) * (A^-1).norm(2)
    print("Conditionnement en norme 2 :", cond)
    bn[-1]=-1/10
    bn[2] = 20
    b = vector(QQ,bn)
    x = A.solve_right(b)

    print("Solution de Ax = b :")
    bb64 = np.array(b)
    bb32 = bb64.astype(np.float32)
    AA64 = np.array(A)
    AA32 = AA64.astype(np.float32)

    x64 = scipy.linalg.solve(AA64, bb64)
    x32 = scipy.linalg.solve(AA32, bb32)

    print("Norme infinie de l'erreur (précision 64 bits) :", scipy.linalg.norm(np.array(x)-x64, np.inf))
    print("Norme infinie de l'erreur (précision 32 bits) :", scipy.linalg.norm(np.array(x)-x32, np.inf))
    print("||erreur|| / cond (précision 64 bits) : ", scipy.linalg.norm(np.array(x)-x64, np.inf)/cond)
    print("||erreur|| / cond (précision 32 bits) : ", scipy.linalg.norm(np.array(x)-x32, np.inf)/cond)

## Matrice du Laplacien

L'exemple de la matrice de Hilbert est un cas extrême permettant de bien mettre en lumière les difficultés associées à une matrice mal conditionnée ; celui de la matrice de Vandermonde, après les chapitres de l'interpolation et de la quadrature, est déjà plus réaliste et on le rencontre en pratique, même s'il y a des moyens d'évaluer les poids de manière exacte et donc de s'affranchir des erreurs associées au mauvais conditionnement et à la résolution du système linéaire.

Un cas classique vu en cours de système linéaire que l'on sera amené à résoudre est celui de la résolution des EDPs par différence finies centrées et de la matrice du Laplacien. Dans ce cas pratique et classique, nous revenons sur le conditionnement de ce type de matrices symétriques définies positives et de leur évaluation par les outils classiques de SciPy. Il apparaît clairement que dans ce cas, le conditionnement augmente de manière quadratique dans le nombre de points et non pas de manière exponentielle ! Considérér des matrices de taille plusieurs centaines d'unité reste donc tout à fait raisonnable avec les outils habituels en double précision.

### Conditionnement en fonction de la taille

In [ ]:
def laplacian(n):
    M = -2*np.identity(n)
    for i in range(1,n):
        M[i,i-1] = 1
        M[i-1,i] = 1
    return M

cond = np.zeros(40)
condinf = np.zeros(40)

for i in range(5, 205, 5):
    print("------------------------------------------------------")
    print("Matrice du laplacien de taille : ", i)

    MM=laplacian(i)
    AA=matrix(QQ,MM)
    
    condinf[(i-4)//5] = AA.norm(Infinity) * (AA^-1).norm(Infinity)
    print("Conditionnement associé à la norme infinie (sage) : ",condinf[(i-4)//5])   
    print("Conditionnement associé à la norme 2 (sage)       : ", AA.norm(2) * (AA^-1).norm(2))
    print("Conditionnement associé à la norme 2 (linalg)     : ", np.linalg.cond(MM))
    cond[(i-4)//5]=(np.sin(i*np.pi/2/(i+1))/np.sin(np.pi/2/(i+1)))**2
    print("Conditionnement exacte                            : ", cond[(i-4)//5]) 


### Estimation du conditionnement

In [ ]:
nn = np.arange(5,205,5)

plt.figure(figsize=[16, 8])
# titre de la figure
plt.title("Conditionnement matrice de discrétisation du Laplacien")
# titre des axes
plt.xlabel('taille matrice')
plt.ylabel('Conditionnement')
# limites des axes
#plt.axis([0, n, np.min(np.abs(eig_val.real)), 10.])
plt.plot(nn, cond, color = 'blue',label='Conditionnement exact')
plt.plot(nn, 4*(nn+1)**2/np.pi**2, '--', linewidth=2,color = 'red',label='(2(N+1)/pi)^2')
plt.plot(nn, condinf , ':', linewidth=2,color = 'green',label='Conditionnement infini')
plt.legend(loc='upper left', frameon=False)
plt.show()

Dans le cadre de la matrice du Laplacien avec conditions de Dirichlet sur l'intervalle $[0,1]$, on peut obtenir une estimation très précise du conditionnement en norme $2$ : $4(n+1)^2/\pi^2$, comme le montre le graphique ci-dessus.